In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

Building model here

In [3]:
# df_comb.drop(['V14', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21'], inplace=True, axis=1)
# df_comb.drop(['V11', 'V14', 'V16', 'V17', 'V20', 'V21'], inplace=True, axis=1)

- Tried to create some additional columns based on date column

In [5]:
df_tr = pd.read_csv('mod_train.csv') #using my modified train dataset after clustering some columns in the test_helper dataset and using that as a feature in train dataset, might prove to be a bad move!

## creating additional columns ##

# df_tr['V5_yearday'] = pd.to_datetime(df_tr['V5']).dt.day_of_year
df_tr['V5_month'] = pd.to_datetime(df_tr['V5']).dt.month
df_tr['V5_day'] = pd.to_datetime(df_tr['V5']).dt.day
# df_tr['V5_hour'] = pd.to_datetime(df_tr['V5']).dt.hour
# df_tr['V5_minute'] = pd.to_datetime(df_tr['V5']).dt.minute
# df_tr['V13_yearday'] = pd.to_datetime(df_tr['V13']).dt.day_of_year
# df_tr['V13_month'] = pd.to_datetime(df_tr['V13']).dt.month
# df_tr['V13_day'] = pd.to_datetime(df_tr['V13']).dt.day

In [6]:
# df_tr.dropna(inplace=True)

In [7]:
columns_encode = ['V2', 'V8', 'V9', 'V10', 'V12']
columns_drop = ['V1', 'V3', 'V5', 'V11', 'V13', 'Target']

In [8]:
x_tr = df_tr.drop(columns_drop, axis=1)
y_tr = df_tr['Target']

#random bullshit go
x_tr.loc[x_tr['V7'].eq('A'), 'V7'] = 1
x_tr.loc[x_tr['V7'].eq('B'), 'V7'] = 2
x_tr.loc[x_tr['V7'].eq('C'), 'V7'] = 3
x_tr.loc[x_tr['V7'].eq('D'), 'V7'] = 4
x_tr.loc[x_tr['V7'].eq('E'), 'V7'] = 5
x_tr.loc[x_tr['V7'].eq('F'), 'V7'] = 6
x_tr.loc[x_tr['V7'].eq('G'), 'V7'] = 7
x_tr.loc[x_tr['V7'].eq('H'), 'V7'] = 8
x_tr.loc[x_tr['V7'].eq('I'), 'V7'] = 9
x_tr.loc[x_tr['V7'].eq('J'), 'V7'] = 10
x_tr['V7'] = x_tr['V7'].astype(int)

In [9]:
x_tr.drop('Unnamed: 0', axis=1, inplace=True)

- feel free to experiment with encoders, count encoder is ok for starting but not ideal in this case

In [10]:
# encoder = ce.OrdinalEncoder(cols = columns_encode)
encoder = ce.CountEncoder(cols=columns_encode)
# encoder = ce.BinaryEncoder(cols=columns_encode)
tr_encoded = encoder.fit_transform(x_tr, y_tr)

In [11]:
# tr_encoded.drop(['V2_25','V2_75'], inplace=True, axis=1)
tr_encoded.drop(['V2'], inplace=True, axis=1)

- why random_state = 42? because it's the answer to life, universe and everything! also test_size = 0.35 was giving the most accurate result of what i could get

In [12]:
x_tr_n, x_val, y_tr_n, y_val = train_test_split(tr_encoded, y_tr, test_size=0.35, random_state=42)

- random forest and logistic classifiers can't handle NaN values and we have a LOT of that

In [13]:
import xgboost as xgb
# import lightgbm as lgb

model = xgb.XGBClassifier()
# model = lgb.LGBMClassifier()
model.set_params(n_estimators=100, max_depth=5)

# model = RandomForestClassifier()

model.fit(x_tr_n, y_tr_n)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [14]:
tr_score = model.score(x_tr_n, y_tr_n)

In [15]:
tr_score

0.9986566662935185

In [16]:
# fraud_prob = model.predict_proba(x_tr)[:,1]
# thresh = 0.95
# y_tr_predict = (fraud_prob > thresh).astype(int)

- using same technique for test data, column count needs to be same for a model to be applied

In [17]:
test_df = pd.read_csv('mod_test.csv')
test_df['Target'] = np.random
# test_df['V5_yearday'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.day_of_year
test_df['V5_month'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.month
test_df['V5_day'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.day
# test_df['V5_hour'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.hour
# test_df['V5_minute'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.minute
# test_df['V13_yearday'] = pd.to_datetime(test_df['V13'], dayfirst=1).dt.day_of_year
# test_df['V13_month'] = pd.to_datetime(test_df['V13'], dayfirst=1).dt.month
# test_df['V13_day'] = pd.to_datetime(test_df['V13'], dayfirst=1).dt.day

In [18]:
# test_df.dropna(inplace=True)

In [19]:
x_test = test_df.drop(columns_drop, axis=1)
#random bullshit go AGAIN!
x_test.loc[x_test['V7'].eq('A'), 'V7'] = 1
x_test.loc[x_test['V7'].eq('B'), 'V7'] = 2
x_test.loc[x_test['V7'].eq('C'), 'V7'] = 3
x_test.loc[x_test['V7'].eq('D'), 'V7'] = 4
x_test.loc[x_test['V7'].eq('E'), 'V7'] = 5
x_test.loc[x_test['V7'].eq('F'), 'V7'] = 6
x_test.loc[x_test['V7'].eq('G'), 'V7'] = 7
x_test.loc[x_test['V7'].eq('H'), 'V7'] = 8
x_test.loc[x_test['V7'].eq('I'), 'V7'] = 9
x_test.loc[x_test['V7'].eq('J'), 'V7'] = 10
x_test['V7'] = x_test['V7'].astype(int)

In [20]:
x_test.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
test_encoded = encoder.transform(x_test)

In [22]:
# test_encoded.drop(['V2_25','V2_75'], axis=1, inplace=True)
test_encoded.drop(['V2'], axis=1, inplace=True)

In [23]:
prediction = model.predict(test_encoded)
probab = model.predict_proba(test_encoded)[:,1]

In [24]:
a = prediction[prediction>0]

In [25]:
len(a)

2786

- this is the required output format

In [26]:
results_df = pd.DataFrame({
    'V2': test_df['V2'],
    'Probability': probab,
    'Target': prediction
})

In [27]:
results_df.to_csv('result.csv')